In [2]:
import re,os,glob2

def convertToRegex(path,prePathRegex):
    (folder,fileName) = os.path.split(path);
    mo = re.match(prePathRegex, path);
    if(mo):
        s = mo.groups(1)[0];
        fileName2 = fileName.replace(".",r"\."); # replace dots
        replaceString = [
                regexFunc(fileName2,s)
            ]
        #print("Change :", fileName,"-->", s)
        return fileName,replaceString;
        
    else:
        raise NameError("No Match: ", mo)
        
def makeFoldersSubCaps(globbingExp,exceptFolder,exceptRename,relTo):
    renames=[]
    folders = []
    for glob in globbingExp:
        folders += glob2.glob(glob)
        
    # filter with regex
    for reg in exceptFolder:
        l = lambda f : not re.match(reg,os.path.realpath(f))
        folders= list(filter( l , folders))
    
    folders = [ os.path.relpath(f, relTo).split("/") for f in folders ]
    # sort longest path first because we mv folders
    folders.sort(key= lambda x: len(x))
    folders.reverse()
    
    
    for dirs in folders:
            
        # loop over all dirs
        origPath= os.path.join(*dirs)
        # change last dir
        change=True
        for reg in exceptRename:
            m = re.match(reg,dirs[-1])
            if m:
                change = False
                break;
        if change:
            dirs[-1] = dirs[-1][0].lower() + dirs[-1][1:]

        newPath = os.path.join(*dirs)
            
        if  origPath != newPath :
            renames.append("git mv %s %s" % (origPath, newPath))
            
    return renames

In [4]:
def makeReplacementsIncludes(globbingExp, func=convertToRegex):
    replacements =[];
    
    files = []
    for glob in globbingExp:
        files += glob2.glob(glob)
        
    for f in files:
        fileName, RegexS = func(f,".*?(?:src|include)/(.*)");
        replacements += RegexS;
        
    return replacements


def makeReplacementsIncludeGuard(globbingExp):
    commands =[];
    
    files = []
    for glob in globbingExp:
        files += glob2.glob(glob)
        
    for f in files:
            m = re.match(".*?include/(.*)",f)
            if not m:
                raise NameError("No match for file: %s" % f)
                
            newInclude = (m.groups(1)[0]).replace(".","_").replace("/","_")
                
            regex = r'"s@#\s*ifndef\s*(\w*)\s*\n\s*#\s*define\s*(\w*)@#ifndef %s\n#define %s@s"' % (newInclude, newInclude)
            
            
            command = r'fileRegexReplace -f -r %s %s' % (regex, os.path.abspath(f))
            commands.append(command)
    return commands

Granular RigidBody Simulation Framework
===

In [7]:
folder = "../../../"

In [6]:
# rename all folders to smallcaps
globs = [folder + "**/"]


commands = makeFoldersSubCaps(globs,
                              exceptFolder=[".*?media",".*?external",".*?Doc",".*?Simulations"], 
                              exceptRename=["external", "GRSF", "LMatrixGeneration"], 
                              relTo="../")

f = open("../MoveFolders.sh",'tw');
f.write("#!/bin/bash\nshopt -s expand_aliases\nsource ~/.bash_aliases\n")
for c in commands:
    f.write(c + "\n");
f.close();

In [5]:
regexFunc = lambda fileName2,s : "s@%s@%s@g" % (r'#include\s*(?:\"|<)(?!ApproxMVBB).*?/'+fileName2+r'(?:\"|>)', r'#include \"' + s + r'\"')


globs = [folder + "common/include/**/*.*", 
        folder + "projects/granularRigidBodySimulation/include/**/*.*",
        folder + "projects/granularRigidBodySimulationMPI/include/**/*.*",
        folder + "projects/granularRigidBodySimulationGUI/include/**/*.*",
        folder + "projects/converter/include/**/*.*"]

# #include replacements
replacements = makeReplacementsIncludes(globs)
#print(replacements)
s = [r'-r "%s" ' %s + "  \\\n" for s in replacements]
regex = " ".join(s);
command = r'fileRegexReplace -f %s -R ".*\.(hpp|cpp|icc)" -e "*/external/*" ' % regex + "../"

f = open("../../ReplaceIncludes.sh",'tw');
f.write("#!/bin/bash\nshopt -s expand_aliases\nsource ~/.bash_aliases\n")
f.write(command);
f.close();

regexFunc =  lambda fileName2,s : r"s@%s@%s@g" % (r'(src|include)(?!/ApproxMVBB).*?/'+fileName2 , r'\1/' + s )

globs = [folder + "common/include/**/*.*", 
        folder + "projects/granularRigidBodySimulation/include/**/*.*",
        folder + "projects/granularRigidBodySimulationMPI/include/**/*.*",
        folder + "projects/granularRigidBodySimulationGUI/include/**/*.*",
        folder + "projects/converter/include/**/*.*",
        folder + "common/src/**/*.*", 
        folder + "projects/granularRigidBodySimulation/src/**/*.*",
        folder + "projects/granularRigidBodySimulationMPI/src/**/*.*",
        folder + "projects/granularRigidBodySimulationGUI/src/**/*.*",
        folder + "projects/converter/src/**/*.*"]
# cmake replacements (src and includes)
replacements = makeReplacementsIncludes(globs,convertToRegex)
#print(replacements)
s = [r'-r "%s" ' %s + "  \\\n" for s in replacements]
regex = " ".join(s);
command = r'fileRegexReplace -f %s -R ".*(CMakeLists.txt)" -e "*/external/*" ' % regex + "../"

f = open("../../ReplaceIncludesCmake.sh",'tw');
f.write("#!/bin/bash\nshopt -s expand_aliases\nsource ~/.bash_aliases\n")
f.write(command);
f.close();



In [6]:
globs = [folder + "common/include/**/*.*", 
        folder + "projects/granularRigidBodySimulation/include/**/*.*",
        folder + "projects/granularRigidBodySimulationMPI/include/**/*.*",
        folder + "projects/granularRigidBodySimulationGUI/include/**/*.*",
        folder + "projects/converter/include/**/*.*"]

commands = makeReplacementsIncludeGuard(globs)
print(commands)
s = "\n".join(commands);
f = open("../../ReplaceIncludeGuards.sh",'tw');
f.write("#!/bin/bash\nshopt -s expand_aliases\nsource ~/.bash_aliases\n")
f.write(s);
f.close();

['fileRegexReplace -f -r "s@#\\s*ifndef\\s*(\\w*)\\s*\\n\\s*#\\s*define\\s*(\\w*)@#ifndef GRSF_common_DemangleTypes_hpp\\n#define GRSF_common_DemangleTypes_hpp@s" /home/zfmgpu/Desktop/Repository/GRSFramework/common/include/GRSF/common/DemangleTypes.hpp', 'fileRegexReplace -f -r "s@#\\s*ifndef\\s*(\\w*)\\s*\\n\\s*#\\s*define\\s*(\\w*)@#ifndef GRSF_common_CPUTimer_hpp\\n#define GRSF_common_CPUTimer_hpp@s" /home/zfmgpu/Desktop/Repository/GRSFramework/common/include/GRSF/common/CPUTimer.hpp', 'fileRegexReplace -f -r "s@#\\s*ifndef\\s*(\\w*)\\s*\\n\\s*#\\s*define\\s*(\\w*)@#ifndef GRSF_common_SerializationHelpersTuple_hpp\\n#define GRSF_common_SerializationHelpersTuple_hpp@s" /home/zfmgpu/Desktop/Repository/GRSFramework/common/include/GRSF/common/SerializationHelpersTuple.hpp', 'fileRegexReplace -f -r "s@#\\s*ifndef\\s*(\\w*)\\s*\\n\\s*#\\s*define\\s*(\\w*)@#ifndef GRSF_common_GetFileDescriptorInfo_hpp\\n#define GRSF_common_GetFileDescriptorInfo_hpp@s" /home/zfmgpu/Desktop/Repository/GRSFra

In [5]:
## replace CmakeList.files
fileRegexReplace -r 's@\$\{(\w*)\}/(include|src)/@\${\1}/\2/GRSF/@g' -R ".*CMakeLists.txt" -e ".*external.*" \
/home/zfmgpu/Desktop/Repository/SimulationFramework/SourceCode/Projects/SimulationFramework


fileRegexReplace -r 's@\$\{(\w*)\}/(include|src)/@\${\1}/\2/GRSF/@g' -R ".*IncludeModulesSimulationFramework.cmake" -e ".*external.*" \
/home/zfmgpu/Desktop/Repository/SimulationFramework/SourceCode/Projects/

SyntaxError: invalid syntax (<ipython-input-5-979e1449f35e>, line 2)

In [38]:
globs = [folder + "common/include/**/*.*", 
        folder + "projects/granularRigidBodySimulation/include/**/*.*",
        folder + "projects/granularRigidBodySimulationMPI/include/**/*.*",
        folder + "projects/granularRigidBodySimulationGUI/include/**/*.*",
        folder + "projects/converter/include/**/*.*",
        folder + "common/src/**/*.*", 
        folder + "projects/granularRigidBodySimulation/src/**/*.*",
        folder + "projects/granularRigidBodySimulationMPI/src/**/*.*",
        folder + "projects/granularRigidBodySimulationGUI/src/**/*.*",
        folder + "projects/converter/src/**/*.*"]

files =[]
for g in globs:
    files += glob2.glob(g)



replace = [ 
            [".*?/GRSFramework/projects/granularRigidBodySimulation/", "[SIM]/" ] ,
            [".*?/GRSFramework/projects/granularRigidBodySimulationGUI/", "[GUI]/" ],
            [".*?/GRSFramework/projects/granularRigidBodySimulationMPI/", "[MPI]/" ],
            [".*?/GRSFramework/projects/converter/", "[CV]/" ],
            [".*?/GRSFramework/common/","[CS]/"] ,
            [".*?/ApproxMVBB/", "[A]/" ]
          ]

#make regex list
regexList = []
for i,f in enumerate(files):
    files[i] = f= os.path.abspath(f)
    
    # replace main folders
    for r in replace:
        s=re.sub(r[0],r[1],f)
        if s!=f:
            files[i] = f = s
            break
    
    base,ext = os.path.splitext(f)
    l = f.split("/")
    t = "".join([ r"(?:" + s + "/)?" for s in l[:-1] ] + [l[-1].replace(".","\.")])
    
    regex = r"\&\s*\\\path{.*?}(.*?)\%\s*regex:\s*"+ r"(%s)"   %  t
    repl  = r"&\\\path{%s}" % f + r"\1% regex:\2"
    
    regexList += [[regex,repl]]


s = [r'-r "s@%s@%s@g" ' % (reg[0],reg[1]) + "  \\\n" for reg in regexList]
regex = " ".join(s);
command = r'fileRegexReplace -f %s' % regex + "file.txt"
f = open("../../ReplaceThesis.sh",'tw');
f.write("#!/bin/bash\nshopt -s expand_aliases\nsource ~/.bash_aliases\n")
f.write(command);
f.close();


ApproxMVBB
====

In [ ]:
dirs = ["/home/zfmgpu/Desktop/Repository/ApproxMVBB/include",
       "/home/zfmgpu/Desktop/Repository/ApproxMVBB/src",
       "/home/zfmgpu/Desktop/Repository/ApproxMVBB/external/Diameter/include",
        "/home/zfmgpu/Desktop/Repository/ApproxMVBB/external/GeometryPredicates/include",
       "/home/zfmgpu/Desktop/Repository/ApproxMVBB/external/pugixml/include",
       "/home/zfmgpu/Desktop/Repository/ApproxMVBB/external/meta/include"]
commands = makeReplacementsIncludeGuard(dirs)
print(commands)
s = "\n".join(commands);
f = open("../ReplaceIncludeGuards.sh",'tw');
f.write("#!/bin/bash\nshopt -s expand_aliases\nsource ~/.bash_aliases\n")
f.write(s);
f.close();

In [ ]:
dirs = ["../../../MinimumVolumeBox/external/Diameter/include"]
        
replacements = makeReplacements(dirs)
print(replacements)
s = [r'-r "%s"' %s for s in replacements]
regex = " ".join(s);

command = r'fileRegexReplace -f %s -R ".*\.(hpp|cpp|icc)" -e "*/external/*" /home/zfmgpu/Desktop/Repository/SimulationFramework/SourceCode/Projects/MinimumVolumeBox/'  % regex
f = open("../ReplaceIncludes.sh",'tw');
f.write("#!/bin/bash\nshopt -s expand_aliases\nsource ~/.bash_aliases\n")
f.write(command);
f.close();


In [ ]:
fileRegexReplace -r 's@\$\{(\w*)\}/(include|src)/@\${\1}/\2/GMSF/@g' -R ".*CMakeLists.txt" -e ".*external.*" \
/home/zfmgpu/Desktop/Repository/SimulationFramework/SourceCode/Projects/MinimumVolumeBox/


CudaFramework
===

In [ ]:
dirs = ["/home/zfmgpu/Desktop/Repository/CudaFramework/include/"]
        
replacements = makeReplacements(dirs)
print(replacements)
s = [r'-r "%s"' %s for s in replacements]
regex = " \\\n".join(s);

f = open("../ReplaceIncludes.sh",'tw');
f.write("#!/bin/bash\nshopt -s expand_aliases\nsource ~/.bash_aliases\n")

command = r'fileRegexReplace -f %s -R ".*\.(hpp|cpp|icc|cu|cuh)" -e "*/external/*" /home/zfmgpu/Desktop/Repository/CudaFramework/src'  % regex
f.write(command);
f.write('\n')
command = r'fileRegexReplace -f %s -R ".*\.(hpp|cpp|icc|cu|cuh)" -e "*/external/*" /home/zfmgpu/Desktop/Repository/CudaFramework/include'  % regex
f.write(command);

command = r'fileRegexReplace -f %s -R ".*\.(hpp|cpp|icc|cu|cuh)" -e "*/external/*" /home/zfmgpu/Desktop/Repository/CudaFramework/projects'  % regex
f.write(command);

f.close();

In [ ]:
commands = makeReplacementsIncludeGuard(dirs)
print(commands)
s = "\n".join(commands);
f = open("../ReplaceIncludeGuards.sh",'tw');
f.write("#!/bin/bash\nshopt -s expand_aliases\nsource ~/.bash_aliases\n")
f.write(s);
f.close();